In [1]:
import wikipedia
import pandas as pd
from bs4 import BeautifulSoup
import re
from  extract_book_features import extract_book_features

# wikipedia.set_lang("en")

In [2]:
input_folder = "../generated/"
output_folder = input_folder

In [3]:
books = pd.read_csv(input_folder + 'books2.csv')
books

,Wikipedia ID,Freebase ID,Book Title,Author,Publication Year,Genres,Summary,Rating Count,Average Rating,Number of Pages,Best Seller,Film,Saga,Followed by,Preceded by
0,7956168,/m/026lcx6,The Kitchen God's Wife,Amy Tan,1991.0,NaN,The Kitchen God's Wife opens with the narrati...,116.0,8.280000,4151.0,No,Yes,No,NaN,NaN
1,1703697,/m/05pr53,The Testament,John Grisham,1999.0,"['Thriller', 'Fiction', 'Suspense']","Troy Phelan, an eccentric elderly billionaire...",261.0,7.544643,330.0,No,No,No,NaN,NaN
2,999996,/m/03y389,Airframe,Michael Crichton,1996.0,"['Science Fiction', 'Novel']",The novel opens aboard Hong Kong-based Transp...,129.0,7.719298,352.0,No,No,No,NaN,NaN
3,1298672,/m/04qyx7,Timeline,Michael Crichton,1999.0,"['Science Fiction', 'Fiction', 'Suspense']","In the middle of the New Mexico desert, a vac...",263.0,8.052083,464.0,No,Yes,No,Airframe,Prey
4,73408,/m/0jqbz,To Kill a Mockingbird,Harper Lee,1960.0,['Fiction'],The book opens with the Finch family's ancest...,267.0,9.043165,281.0,Yes,Yes,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4732,4760716,/m/0clxxp,The Game-Players of Titan,Philip K. Dick,1963.0,"['Science Fiction', 'Speculative fiction', 'Fa...","Pete Garden, the protagonist, is one of sever...",1.0,7.000000,191.0,No,No,No,NaN,NaN
4733,480566,/m/02fmjq,Galactic Pot-Healer,Philip K. Dick,1969.0,"['Science Fiction', 'Speculative fiction', 'Fi...",The novel takes place in a dismal future Amer...,1.0,8.000000,144.0,No,No,No,NaN,NaN
4734,2497286,/m/07h_l2,Solar Lottery,Philip K. Dick,1955.0,"['Science Fiction', 'Speculative fiction', 'Fi...",Solar Lottery takes place in a world dominate...,1.0,7.000000,188.0,No,No,No,NaN,NaN
4735,23616820,/m/06zktr5,Le Sang noir,Louis Guilloux,1935.0,NaN,"One day in 1917 an aging philosophy tutor, ni...",1.0,NaN,NaN,No,Yes,No,NaN,NaN


In [4]:
books['Saga'].value_counts()

Saga
No                              3552
Nero Wolfe                        22
James Bond                        15
Lord Peter Wimsey                 12
Heinlein juveniles                12
                                ... 
Adam DalglieshNo. 1                1
The Extraordinary Voyages#11       1
Inspector Wexford# 15              1
Adam Dalgliesh#2                   1
The Matador Series                 1
Name: count, Length: 671, dtype: int64

Aquí faltaría generar un dataset de sagas.

In [5]:
sagas = pd.DataFrame(books['Saga'].unique(), columns=['Saga'])
sagas
# sagas.to_csv(output_folder + 'sagas.csv', index=False)

,Saga
0,No
1,in Death
2,Jack Ryan
3,Alex Cross
4,The Wicked Years
...,...
667,Dan Starkeynovels
668,Greenwich Village Trilogy
669,The Underland Chronicles
670,Novaya volna russkoy fantastiki


In [6]:
authors = pd.DataFrame(books['Author'].unique(), columns=['Author'])
# Si se para la ejecución, hay que cargar los datos para no sobreescribirlos.
# authors = pd.read_csv(input_folder + 'authors100.csv')
# authors

In [7]:
books_titles = list(books['Book Title'].values)
# new_books100 = pd.read_csv('../clean_data/new_books100.csv')
# books_titles += list(new_books100['Book Title'].values)
books_titles

["The Kitchen God's Wife",
 'The Testament',
 'Airframe',
 'Timeline',
 'To Kill a Mockingbird',
 'Pigs in Heaven',
 'Pride and Prejudice',
 'From the Corner of His Eye',
 'Purity in Death',
 'The Street Lawyer',
 'Breathing Lessons',
 'The Joy Luck Club',
 'The Tao of Pooh',
 'The Prince',
 'Starship Troopers',
 "Anil's Ghost",
 'Turning Thirty',
 'Decipher',
 'The Catcher in the Rye',
 "Pretend You Don't See Her",
 'The Pillars of the Earth',
 'The Five People You Meet in Heaven',
 'Blood Oath',
 'The Mosquito Coast',
 'The Sum of All Fears',
 'One Hundred Years of Solitude',
 'Gangster',
 'Rebecca',
 'Contact',
 'The Brimstone Wedding',
 'Puerto Vallarta Squeeze',
 'Wish You Well',
 "The Hitchhiker's Guide to the Galaxy",
 'Fahrenheit 451',
 'The God of Small Things',
 'Four Blind Mice',
 'Congo',
 'Memoirs of a Geisha',
 'The Brethren',
 'The King of Torts',
 'The Rescue',
 'Pet Sematary',
 'Deception Point',
 'Wicked: The Life and Times of the Wicked Witch of the West',
 'Life of 

In [8]:
len(books_titles)

4737

### Detectamos si una página de wikipedia es un libro o es sobre una película o de otro tipo.

Para que la url funcione corregimos algunos acentos. 

Si en el título hay la palabra novel, la función devuelve true.
Si en el título hay la palabra film o videogame, la función devuelve false.

En los otros casos usamos un patron con re para buscar si es un libro.

Usamos beautifulsoup para encontrar elementos de la página.

In [ ]:
def is_book(url, title):
        url = url.replace('/wiki/', '')
        url = url.replace('_', ' ')
        url = url.replace('%27', "'")
        url = url.replace('%C3%', 'ú')
                

        # print(url)
        page = wikipedia.page(title, auto_suggest=False)
       


        soup = BeautifulSoup(page.html(), 'html.parser')
        
        if 'novel' in title:
                return True, soup

        if 'film' in title or 'videogame' in title.replace(" ", ""):
                # print("Not a book")
                return False, None
        
        # is {30 characters} donde no están las palabras based, from o film y acaba en novel o book
        # Funciona razonablemente bien

        pattern = re.compile(r' is  *(?!.{0,30}\b(?:film|based|from)\b).{0,30}(novel|book)')

        match = pattern.search(str(soup.text))

        
        if not match:
                # print("Not a book")
                return False, None
        
        return True, soup

### Extraemos las caracteristicas de los libros encontrados
Si no encontramos una característica, ponemos nan.
La variables que buscamos son genres, summary, pages, best_seller, film, saga, preceded_by y followed_by. (que son las que podemos extraer en wikipedia)

In [ ]:
def preprocess_book(soup):

        infobox_table = soup.find('table', class_='infobox vcard')

        if infobox_table:
                genres = infobox_table.find('th', string='Genre')

                if genres is None:
                        genres = float('nan')
                else:
                        genres = genres.find_next('td').text.split(',')
        else:
                genres = float('nan')

        summary = soup.find('span', string=lambda text: text and 'Plot' in text)

        if summary is None:
                summary = float('nan')
        else:
                summary = summary.find_next('p')
                if summary:
                        summary = summary.text

        values = extract_book_features(soup, infobox_table=infobox_table)

        if not values:
                return None

        [pages, best_seller, film, saga, preceded_by, followed_by] = values


        return [genres, summary, pages, best_seller, film, saga, preceded_by, followed_by]

### Extraemos las características de los autores
Concretamente guardamos su año de nacimiento, que géneros escribio, sus libros.

In [ ]:

def get_author(author, authors):
        page = None
        try:
                page = wikipedia.page(author, auto_suggest=False)
        except:
                pass
        
        if not page:
                try:
                        page = wikipedia.page(author + ("writer"), auto_suggest=False)
                except:
                        pass
        if not page:
                try:
                        page = wikipedia.page(author + ("author"), auto_suggest=False)
                except:
                        pass
        if not page:
                return []



        soup = BeautifulSoup(page.html(), 'html.parser')
        
        infobox_table = soup.find('table', class_='infobox')

        if infobox_table:
                birth = infobox_table.find('th', string='Born')
                
                if birth:
                        birth = birth.find_next('td').text
                        birth_year = re.search(r'\d{4}', birth)
                        if birth_year:
                                birth_year = birth_year.group(0)
                        else:
                                birth_year = float('nan')
                else:
                        birth_year = float('nan')

                genres = infobox_table.find('th', string='Genre')
                
                if genres:
                        genres = genres.find_next('td').text
                        # print(genres)
                else:
                        genres = float('nan')
        else:
                birth_year = float('nan')
                genres = float('nan')


        # Encuentra todos los elementos que coinciden con el patrón

        authors_books = []

        # Has bibliography?     
        bibliography = None
        try:
                bibliography = wikipedia.page(author + " bibliography")
        except:
                pass

        if bibliography:
                bibliography_soup = BeautifulSoup(bibliography.html(), 'html.parser')
                target_elements = bibliography_soup.find_all('li')
        else:        
                target_elements = soup.find_all('li')



        pattern1 = re.compile(r'<li><i><a\s+href="([^"]+)"\s+title="([^"]+)">([^<]+)</a></i> \((\d{4})\)</li>')
        pattern2 = re.compile(r'<li>(\d{4}) <i><a href="([^"]*)" title="([^"]*)">([^<]+)</a></i></li>')
        pattern3 = re.compile(r'<li><i><a\s+href="([^"]+)"\s+title="([^"]+)">([^"]+)</a></i>, ((\d{4}))</li>')

        new_books_local = []

        global new_books
        
        for element in target_elements:

                match1 = pattern1.search(str(element))
                match2 = pattern2.search(str(element))
                match3 = pattern3.search(str(element))

                if not match1 and not match2:
                        continue
                
                if bibliography and match2:
                        book_year = match2.group(1)
                        book_url = match2.group(2)
                        book_title = match2.group(3)


                elif match1:
                        book_url = match1.group(1)
                        book_title = match1.group(2)
                        book_year = match1.group(4)
                elif match3:
                        book_url = match3.group(1)
                        book_title = match3.group(2)
                        book_year = match3.group(4)

                is_book_, book_soup = is_book(book_url, book_title)

                if not is_book_:
                        continue
                
                authors_books.append(book_title)


                if book_title not in books_titles:
                        values = preprocess_book(soup=book_soup)
                        if not values: 
                                genres, summary, pages, best_seller, film, saga, preceded_by, followed_by = [float('nan')] * 8
                        else:
                                [genres, summary, pages, best_seller, film, saga, preceded_by, followed_by] = values
                        rating_count = float('nan')
                        average_rating = float('nan')

                        new_book = {
                                'Book Title': book_title, 
                                'Author': author, 
                                'Publication Year': book_year, 
                                'Genres': genres, 
                                'Summary': summary, 
                                'Rating Count': rating_count, 
                                'Average Rating': average_rating, 
                                'Number of Pages': pages, 
                                'Best Seller': best_seller,
                                'Film': film,
                                'Saga': saga, 
                                'Preceded by': preceded_by, 
                                'Followed by': followed_by
                        }
                        # print(new_book)
                        new_books_local.append(new_book)       
                        books_titles.append(book_title)

        new_books += new_books_local
        # print(f"Author: {author} - Birth Year: {birth_year} - Genres: {genres} - Books: {authors_books}")
        authors.loc[authors['Name'] == author, ['Birth Year']] = birth_year
        authors.loc[authors['Name'] == author, ['Genres']] =  str(genres)
        authors.loc[authors['Name'] == author, ['Books']] = str(authors_books)


### Función para iterar todos los autores

In [ ]:
def get_authors(i):
        for i, row in authors[i:].iterrows():
                print(f"Index: {i}")
                get_author(row['Name'], authors)


In [9]:
new_books = []

get_authors(1778)

# Pruebas
# print(get_author("Mary Wesley", authors, new_books))
# print(get_author('J. R. R. Tolkien', authors, new_books))
# print(get_author('Michael Crichton', authors, new_books))




KeyboardInterrupt



In [ ]:
len(new_books)

2579

In [ ]:
new_books_df = pd.DataFrame(new_books)
new_books_df['Book Title'].unique().shape

(2579,)

In [ ]:
authors

,Name,Birth Year,Genres,Books
0,Amy Tan,1952.0,"[""Children's book""]","['The Joy Luck Club (novel)', ""The Kitchen God..."
1,John Grisham,1955.0,"['Legal thriller', ' young adult']","['A Time to Kill (Grisham novel)', 'Sycamore R..."
2,Michael Crichton,1942.0,NaN,"['The Andromeda Strain', 'The Terminal Man', '..."
3,Harper Lee,1926.0,['Fiction'],"['To Kill a Mockingbird', 'Go Set a Watchman',..."
4,Barbara Kingsolver,1955.0,Historical fiction,[]
...,...,...,...,...
2116,Li Zhisui,1919,nan,[]
2117,Steve Perry,1949,nan,['The Case of the Curious Bride']
2118,E.L. Konigsburg,1930,"Children's novels, short stories, picture books",[]
2119,Louis Guilloux,1899,nan,[]


In [ ]:
# new_books_df.to_csv(f'../clean_data/new_books{200}.csv', index=False)
# authors.to_csv(f'../clean_data/authors{100}.csv', index=False)

#### Testing patterns

In [ ]:
text = "The Terminal Man is a novel "

pattern = re.compile(r'is a *(?!.*\b(?:film|based)\b).{0,100} novel')


match = pattern.search(text)

if match:
    print(0)




text = "The Terminal Man is a film novel"


match = pattern.search(text)

if match:
    print(1)



text = "The Terminal Man is a fdadfa novel by American writer Michael Crichton.[2][3] It is his second novel under his own name and his twelfth overall, and is about the dangers of mind control. It was published in April 1972, and also serialized in Playboy in March, April, and May 1972. In 1974, it was made into a film of the same name."


match = pattern.search(text)

if match:
    print(2)


text = "The Terminal Man is a fdadfa based novel by American writer Michael Crichton.[2][3] It is his second novel under his own name and his twelfth overall, and is about the dangers of mind control. It was published in April 1972, and also serialized in Playboy in March, April, and May 1972. In 1974, it was made into a film of the same name."


match = pattern.search(text)

if match:
    print(3)



    